In [ ]:
from splinter import Browser
from bs4 import BeautifulSoup
import pymongo
import requests

In [ ]:
# browser = Browser('chrome',headless=False)
def init_browser():
    chrome_location = "C:\\Users\\ssank\\chromedriver_win32\\chromedriver.exe"
    chrome_location = chrome_location.replace("\\","/")

    executable_path = {"executable_path":chrome_location}
    return Browser("chrome", **executable_path, headless=False)

In [ ]:
def get_latest_NASA_news():
    browser = init_browser()
    NASA_MARS_url = "https://mars.nasa.gov/news/"
    browser.visit(NASA_MARS_url)
    news_soup = BeautifulSoup(browser.html,"lxml")
    latest_news = news_soup.find_all("div",{"class":"list_text"})[0]
    
    return {
        "news-title":latest_news.find("div",{"class":"content_title"}).text,
        "news-content":latest_news.find("div",{"class":"article_teaser_body"}).text
    }    

In [ ]:
def get_MARS_img():
    browser = init_browser()
    JPL_home_url = "https://www.jpl.nasa.gov"
    JPL_img_url = JPL_home_url+"/spaceimages/?search=&category=Mars"
    browser.visit(JPL_img_url)
    jpl_soup = BeautifulSoup(browser.html,"html.parser")

    items = jpl_soup.find("div",{"class":"carousel_items"})
    img_title = items.find("h1",{"class":"media_feature_title"})
    featured_img = items.find("article")
    img_url = JPL_home_url+featured_img['style'].split(':')[1].split('\'')[1]
    response = requests.get(img_url, stream=True)

    featured_img_name = img_title.text.strip(' \t\n\r')+".jpg"

    if response.status_code == 200:
        with open(featured_img_name, 'wb') as f:
            f.write(response.content)
    return featured_img_name

In [ ]:
def get_MARS_temperature():
    browser = init_browser()
    twitter_report_url = "https://twitter.com/marswxreport?lang=en"
    browser.visit(twitter_report_url)
    temp_soup = BeautifulSoup(browser.html,"html.parser")
    return temp_soup.find("ol",{"id":"stream-items-id"}).find("li").find("p").text